In [11]:
from PIL import Image
import numpy as np
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader   

In [12]:
# Define the execution device 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
# Define a class to convert colored images into grayscale images

class GrayScaleImage():
    
    def __init__(self, images):
        self.images = images
    
    def dataset2gray(self):
        return map(lambda img: img.convert('L'), self.images)

In [8]:
# Test GrayScaleImage class

img_dir = "C:/Alejandro/Fotos/meme2.png"
img_rgb = Image.open(img_dir)
#img_rgb.show()

images = [img_rgb]
test = GrayScaleImage(images)
img_gray = test.dataset2gray()

for i in img_gray:
    i.show()

In [14]:
# Define normalization

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [42]:
# Load the data

# Use GrayScaleImage() to get the black-and-white images

# Gather all the data

# Normalize it
    

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels, filters, kernel_size, stride=1):
        super().__init__()
        
        self.layer1 = nn.Conv2d(in_channels, out_channels=64, kernel_size, stride=2, padding="same")
        self.layer2 = nn.Conv2d(64, out_channels=128, kernel_size, stride=2, padding="same")
        self.layer3 = nn.Conv2d(128, out_channels=256, kernel_size, stride=2, padding="same")
        self.layer4 = nn.Conv2d(256, out_channels=512, kernel_size, padding="same") # stride = 1
        self.layer5 = nn.Conv2d(512, out_channels=1, kernel_size, padding="same")
    
    def forward(self):
        pass

# model = Discriminator()
# model.to(device)

loss_fn = nn.BCELoss(weight=0.5)
optimizer = Adam(model.parameters(), lr=0.0002, weight_decay=0.5)